In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

/tmp/ipykernel_1586/1799943160.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png', 'pdf')


# Exercise 2

<img src='./images/02.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise_2')

2025/04/06 10:25:39 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_02/mlruns/773631873264077024', creation_time=1743922539674, experiment_id='773631873264077024', last_update_time=1743922539674, lifecycle_stage='active', name='Exercise_1', tags={}>

This code integrates **Optuna** for automated hyperparameter optimization with **MLflow** for experiment tracking. It trains a deep learning model on a binary classification task and evaluates its performance using the Area Under the Curve (AUC) metric. The script optimizes hyperparameters such as batch size, learning rate, number of layers, neurons per layer, and activation function **to maximize the AUC score**, ensuring the model achieves superior predictive performance. Key artifacts, metrics, and parameters are logged with MLflow, providing a structured and reproducible workflow for model development and evaluation.

In [ ]:
from sklearn.datasets import make_moons
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import optuna
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import (train_network, accuracy_score_wrapper, 
                f1_score_wrapper, roc_auc_score_micro_wrapper, 
                weight_reset, set_seed)
from torchinfo import summary

/home/spakdel/miniconda3/envs/python3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
random_state = 42
set_seed(random_state)

X_train, Y_train = make_moons(n_samples=8000, noise=0.4, random_state=random_state)
X_valid, Y_valid = make_moons(n_samples=200, noise=0.4, random_state=random_state)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                            torch.tensor(Y_train, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(X_valid, dtype=torch.float32),
                            torch.tensor(Y_valid, dtype=torch.long))

in_features = 2
out_features =2
loss_func = nn.CrossEntropyLoss()
activation_functions = {
'ReLU': nn.ReLU(),
'Tanh': nn.Tanh(),
'LeakyReLU': nn.LeakyReLU(),
'Sigmoid': nn.Sigmoid()
}

In [ ]:
def plot_results(data_df, close=True):
    sns.lineplot(data_df, x='epoch', y='valid AUC', label='valid AUC')
    plt.xlabel('epoch')
    plt.ylabel('valid AUC')
    plt.title('valid AUC')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig
    

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
import optuna


optuna.logging.set_verbosity(optuna.logging.ERROR)
def champion_callback(study, frozen_trial):
    winner = study.user_attrs.get('winner', None)
    if winner is None:
        print(f'Initial trial {frozen_trial.number} achived value: {frozen_trial.value}')
    elif winner != study.best_value and study.best_value:   # second condition is for preventing zero devision
        improvment_percent = (abs(winner - study.best_value) / abs(study.best_value)) * 100
        print(f'Trial {frozen_trial.number} achived value: {frozen_trial.value} with {improvment_percent:.4f}% improvment')
    study.set_user_attr('winner', study.best_value)


In [ ]:
from mlflow.types import Schema, TensorSpec
from mlflow.models import ModelSignature


def objective(trial):
    params = {
    'batch_size': trial.suggest_int('batch_size', 16, 256),
    'device': device,
    'epochs' : epochs,
    # 'optimizer': optimizer.defaults,
    'loss_function': loss_func.__class__.__name__,
    'learning_rate': trial.suggest_float('lr', 1e-8, 1, log=True),
    'hidden_neurons': trial.suggest_int("neuron_per_layer", in_features, 500),
    'layers': trial.suggest_int("hidden_layers", 1, 20),
    'activation': trial.suggest_categorical("activation", list(activation_functions.keys()))
    }
    sequential_layer = [
        nn.Linear(in_features, params['hidden_neurons']),
        activation_functions[params['activation']]
    ]
    for _ in range(params['layers']):
        sequential_layer.append(nn.Linear(params['hidden_neurons'], params['hidden_neurons']))
        sequential_layer.append(activation_functions[params['activation']])
    sequential_layer.append(nn.Linear(params['hidden_neurons'], out_features))

    model = nn.Sequential(*sequential_layer)
    
    # run_name = f'trial_lr_{params["learning_rate"]:.8f}'
    run_name = f'trial: {trial.number}'
    with mlflow.start_run(nested=True, run_name=run_name):

        optimizer = torch.optim.SGD(model.parameters(), lr=params['learning_rate'])
        params['optimizer'] = optimizer.defaults
        mlflow.log_params(params)

        train_dataloader = DataLoader(train_dataset, batch_size=params['batch_size'],shuffle=True)
        valid_dataloader = DataLoader(valid_dataset, batch_size=params['batch_size'])
        
        with open ("model_summary.txt", "w") as f:
            f.write(str(summary(model)))
        mlflow.log_artifact("model_summary.txt")

        model.apply(weight_reset)
        fc_results = train_network(
            model=model,
            loss_func=loss_func,
            train_loader=train_dataloader,
            valid_loader=valid_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            score_funcs={'Acc':accuracy_score_wrapper, 'F1':f1_score_wrapper, 'AUC':roc_auc_score_micro_wrapper },
            device=device,
            checkpont_file_save='model.pth'
        )
        
        input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        signature = ModelSignature(inputs=input_schema, outputs=output_schema)
        mlflow.pytorch.log_model(model, "model", signature=signature)
        mlflow.log_figure(plot_results(fc_results), "valid_AUC.png")
    return  fc_results['valid AUC'].iloc[-1]


In [ ]:
epochs = 20

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, callbacks=[champion_callback])
champion_trial = study.best_trial
print(f"Champion trial: {champion_trial.number} with value {champion_trial.value}")

Epoch: 100%|██████████| 20/20 [00:26<00:00,  1.30s/it]
2025/04/07 08:19:23 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/07 08:19:34 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Initial trial 0 achived value: 0.12995


Epoch: 100%|██████████| 20/20 [00:13<00:00,  1.48it/s]
2025/04/07 08:19:47 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/07 08:19:51 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Trial 1 achived value: 0.72805 with 82.1510% improvment


Epoch: 100%|██████████| 20/20 [00:20<00:00,  1.02s/it]
2025/04/07 08:20:12 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/07 08:20:16 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Epoch: 100%|██████████| 20/20 [00:12<00:00,  1.57it/s]
2025/04/07 08:20:28 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged 

...

Trial 11 achived value: 0.9301 with 0.0753% improvment


Epoch: 100%|██████████| 20/20 [00:24<00:00,  1.24s/it]
2025/04/07 08:25:41 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/07 08:25:45 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Trial 12 achived value: 0.9320999999999999 with 0.2146% improvment


Epoch: 100%|██████████| 20/20 [00:19<00:00,  1.04it/s]
2025/04/07 08:26:04 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/07 08:26:10 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
Epoch: 100%|██████████| 20/20 [00:23<00:00,  1.18s/it]
2025/04/07 08:26:34 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged 

Champion trial: 12 with value 0.9320999999999999
